In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from os import listdir
from os.path import isfile, join

In [2]:


data_path = '../data/'
import_csv_name = [f for f in listdir(data_path) if (isfile(join(data_path, f))) and (f.endswith('.xlsx'))]

In [3]:
stock_df = pd.DataFrame()

for iter in range(len(import_csv_name)):
    df = pd.read_excel(data_path + import_csv_name[iter])
    df["sector"] = import_csv_name[iter].split('.')[0].replace(" ","")
    required_columns = [item for item in df.columns  if "(SPX Index)" not in item]
    temp_sector_df = df[required_columns]
    temp_sector_df.columns = [item.split('\n')[0] for item in temp_sector_df.columns]
    stock_df = pd.concat([stock_df, temp_sector_df], axis=0, ignore_index=True)
    
stock_df["Date"] = stock_df["Date"].apply(lambda x: x.strftime('%Y%m%d'))
stock_df.fillna(method="ffill",inplace=True)
stock_df.columns = [item.replace(" ","") for item in stock_df.columns]


# scale every column using min-max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
stock_df[stock_df.columns[1:-1]] = scaler.fit_transform(stock_df[stock_df.columns[1:-1]])

# oen hot encoding for sector
stock_df = pd.get_dummies(stock_df, columns=["sector"])
stock_df['Date'] = pd.to_datetime(stock_df['Date'], format='%Y%m%d')

# stock_df.to_csv("../out/scope2_processed_data.csv", index=False)

# Convert date columns to datetime objects
covid = pd.read_csv("../data/covid/covid.csv")
covid = covid[["formatted_date","new_cases"]]
covid[covid.columns[1:]] = scaler.fit_transform(covid[covid.columns[1:]])
covid['formatted_date'] = pd.to_datetime(covid['formatted_date'], format='%Y%m%d')

# Create a lagged version of the 'new_cases' column
covid['new_cases_lagged'] = covid['new_cases'].shift(1)
covid["new_cases_lagged"] = covid["new_cases_lagged"].fillna(0)

# Merge the stock_df and covid DataFrames on the date columns
merged_df = stock_df.merge(covid, left_on='Date', right_on='formatted_date', how='inner')
merged_df.drop(columns=['formatted_date'], inplace=True)



In [4]:
merged_df

,Date,LastPrice,PriceEarningsRatio,ReturnonCapital,Volatility30Day,CurrentMarketCap,RSI14Day,OverridableRawBeta,sector_CommunicationServices,sector_ConsumerDiscretion,...,sector_Energy,sector_Financial,sector_HealthCare,sector_IT,sector_Industrial,sector_Material,sector_RealEstate,sector_Utilities,new_cases,new_cases_lagged
0,2021-12-31,0.043681,0.004251,0.629782,0.096395,0.327993,0.461527,0.495186,1,0,...,0,0,0,0,0,0,0,0,0.374794,0.307790
1,2021-12-31,0.496008,0.008221,0.649853,0.132706,0.429387,0.516572,0.529171,0,1,...,0,0,0,0,0,0,0,0,0.374794,0.307790
2,2021-12-31,0.224548,0.003956,0.696493,0.076320,0.186698,0.757879,0.387721,0,0,...,0,0,0,0,0,0,0,0,0.374794,0.307790
3,2021-12-31,0.095963,0.003499,0.450666,0.188861,0.053804,0.483654,0.932078,0,0,...,1,0,0,0,0,0,0,0,0.374794,0.307790
4,2021-12-31,0.172502,0.001428,0.534820,0.131034,0.329475,0.492837,0.808226,0,0,...,0,1,0,0,0,0,0,0,0.374794,0.307790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7568,2022-01-03,0.175226,0.001471,0.534820,0.131369,0.334204,0.559932,0.808552,0,0,...,0,1,0,0,0,0,0,0,0.239393,0.372942
7569,2022-01-03,0.501754,0.003715,0.674149,0.050572,0.420288,0.634659,0.454097,0,0,...,0,0,1,0,0,0,0,0,0.239393,0.372942
7570,2022-01-03,0.252494,0.006087,0.576702,0.110413,0.233487,0.519024,0.696825,0,0,...,0,0,0,0,1,0,0,0,0.239393,0.372942
7571,2022-01-03,0.993026,0.006945,0.974755,0.161018,0.993193,0.586625,0.665285,0,0,...,0,0,0,1,0,0,0,0,0.239393,0.372942


In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Assuming 'merged_df' is the merged DataFrame from the previous step

# Select the input variables: COVID contraction rates (new_cases_lagged) and stock market sectors
X = merged_df[['new_cases_lagged', 'PriceEarningsRatio', 'RSI14Day',
               'sector_CommunicationServices', 'sector_ConsumerDiscretion',
               'sector_ConsumerStaple', 'sector_Energy', 'sector_Financial',
               'sector_HealthCare', 'sector_IT', 'sector_Industrial',
               'sector_Material', 'sector_RealEstate', 'sector_Utilities']]

# Add a constant term to the predictors (required for statsmodels)
X = sm.add_constant(X)

# Select the output variables: Sector performance metrics (ReturnonCapital, Volatility30Day)
y = merged_df[['ReturnonCapital']]

# Split the dataset into training and testing sets, maintaining the time order of the data
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create a linear regression model
model = sm.OLS(y_train, X_train)

# Fit the model to the training data
results = model.fit()

# Print the summary
print(results.summary())

# Make predictions using the test set
y_pred = results.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse:.2f}")

# Calculate the R-squared score
r2 = r2_score(y_test, y_pred)
print(f"R-squared score: {r2:.2f}")

# Conclusion:
# Summarize the findings and discuss any insights gained from the analysis. Address any limitations of the study and suggest possible improvements or future research directions.


                            OLS Regression Results                            
Dep. Variable:        ReturnonCapital   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     1492.
Date:                Sun, 23 Apr 2023   Prob (F-statistic):               0.00
Time:                        22:26:43   Log-Likelihood:                 6610.3
No. Observations:                6058   AIC:                        -1.319e+04
Df Residuals:                    6044   BIC:                        -1.310e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           